In [11]:
# ============================================================================
# STAGE 1: IMPORTS AND ENVIRONMENT SETUP
# ============================================================================

# Core Data Processing
import pandas as pd
import numpy as np
import os
import warnings
import re
import json
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Union
from dataclasses import dataclass
import pickle
import joblib

warnings.filterwarnings('ignore')

# Deep Learning Frameworks
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.cuda.amp import autocast, GradScaler  # Mixed precision training
from torchvision import transforms, models
from PIL import Image
import torchvision.transforms.functional as TF

# Advanced Vision Models
import timm  # For state-of-the-art vision architectures
from efficientnet_pytorch import EfficientNet  # Efficient architectures

# Machine Learning & Preprocessing
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, PowerTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import IsolationForest  # Outlier detection

# Feature Engineering
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.decomposition import PCA

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Utilities
import time
from collections import defaultdict, Counter
import logging
from datetime import datetime

# Augmentation (Advanced)
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Model Interpretability
try:
    import shap  # For model explainability
    from captum.attr import IntegratedGradients, Saliency  # For feature attribution
    INTERPRETABILITY_AVAILABLE = True
except ImportError:
    INTERPRETABILITY_AVAILABLE = False
    print("⚠️ Interpretability libraries not available. Install: pip install shap captum")

# ============================================================================
# DEVICE SETUP & OPTIMIZATION
# ============================================================================

# Set device with fallback
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'✓ Using GPU: {torch.cuda.get_device_name(0)}')
    print(f'  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')
elif torch.backends.mps.is_available():  # Apple Silicon
    device = torch.device('mps')
    print(f'✓ Using Apple MPS (Metal Performance Shaders)')
else:
    device = torch.device('cpu')
    print(f'⚠️ Using CPU (training will be slower)')

# Performance optimizations
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True  # Auto-tune convolution algorithms
    torch.backends.cudnn.deterministic = False  # Faster but non-deterministic
    torch.backends.cudnn.enabled = True
    print(f'✓ CUDA optimizations enabled')
    
    # Check for mixed precision support
    if torch.cuda.get_device_capability()[0] >= 7:  # Volta or newer
        print(f'✓ Mixed precision training (FP16) supported')
        USE_AMP = True
    else:
        print(f'⚠️ GPU does not support efficient mixed precision')
        USE_AMP = False
else:
    USE_AMP = False

# ============================================================================
# REPRODUCIBILITY SETUP
# ============================================================================

SEED = 42

def set_seed(seed: int = SEED):
    """Set seeds for reproducibility across all libraries"""
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    # Python hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    # Additional reproducibility (slower but deterministic)
    if torch.cuda.is_available():
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(SEED)
print(f'✓ Random seed set to {SEED}')

# ============================================================================
# LOGGING SETUP
# ============================================================================

def setup_logging(log_dir: str = 'logs'):
    """Setup logging for training monitoring"""
    os.makedirs(log_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    log_file = os.path.join(log_dir, f'training_{timestamp}.log')
    
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger(__name__)

logger = setup_logging()
logger.info('='*80)
logger.info('House Price Prediction - Multi-Modal Deep Learning')
logger.info('='*80)

# ============================================================================
# CONFIGURATION CLASS
# ============================================================================

@dataclass
class Config:
    """Configuration for the entire pipeline"""
    
    # Paths
    data_dir: str = 'data'
    image_dir: str = 'data/images'
    excel_path: str = 'data/properties.xlsx'
    output_dir: str = 'outputs'
    model_save_dir: str = 'models'
    
    # Model Architecture
    image_model: str = 'efficientnet_b4'  # Options: efficientnet_b0-b7, resnet50, convnext_tiny
    pretrained: bool = True
    freeze_backbone_epochs: int = 5  # Freeze feature extractor initially
    
    # Training Hyperparameters
    batch_size: int = 16
    num_epochs: int = 100
    learning_rate: float = 1e-4
    weight_decay: float = 1e-4
    early_stopping_patience: int = 15
    gradient_clip: float = 1.0
    
    # Image Processing
    img_size: int = 384  # Higher resolution for better feature extraction
    
    # Data Split
    test_size: float = 0.15
    val_size: float = 0.15
    
    # Cross-Validation
    use_kfold: bool = True
    n_folds: int = 5
    
    # Mixed Precision
    use_amp: bool = USE_AMP
    
    # Ensemble
    use_ensemble: bool = True
    num_ensemble_models: int = 3
    
    def __post_init__(self):
        """Create necessary directories"""
        for dir_path in [self.output_dir, self.model_save_dir, 'logs']:
            os.makedirs(dir_path, exist_ok=True)

# Initialize configuration
config = Config()
logger.info(f'Configuration initialized')

# ============================================================================
# DISPLAY SYSTEM INFO
# ============================================================================

print("\n" + "="*80)
print("SYSTEM INFORMATION")
print("="*80)
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"cuDNN Version: {torch.backends.cudnn.version()}")
print(f"Device: {device}")
print(f"Number of CPU cores: {os.cpu_count()}")
print("="*80 + "\n")

2025-11-26 11:54:04,743 - INFO - ================================================================================
2025-11-26 11:54:04,743 - INFO - House Price Prediction - Multi-Modal Deep Learning
2025-11-26 11:54:04,743 - INFO - ================================================================================
2025-11-26 11:54:04,743 - INFO - Configuration initialized


✓ Using GPU: NVIDIA GeForce RTX 4060
  Memory: 8.59 GB
✓ CUDA optimizations enabled
✓ Mixed precision training (FP16) supported
✓ Random seed set to 42

SYSTEM INFORMATION
PyTorch Version: 2.7.1+cu118
CUDA Available: True
CUDA Version: 11.8
cuDNN Version: 90100
Device: cuda
Number of CPU cores: 24



Stage 2: Load Your Data

In [12]:
# Load and preprocess data
csv_file = 'C:/Users/LEGION/Desktop/big_data/property_cleaned.csv'
print(f'Loading data from: {csv_file}')

df = pd.read_csv(csv_file)
print(f'✓ Loaded {len(df)} rows from cleaned CSV')

# Display columns info
print(f'\nColumns in dataset: {df.columns.tolist()}')

# Prepare image paths
image_dir = Path('C:/Users/LEGION/Desktop/big_data/images')
print(f'Image directory: {image_dir}')

# Check if images exist and create image paths
df['image_path'] = df['id'].apply(lambda x: image_dir / f'{x}.jpg')
print(f'Created image paths for {len(df)} properties')


Loading data from: C:/Users/LEGION/Desktop/big_data/property_cleaned.csv


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/LEGION/Desktop/big_data/property_cleaned.csv'

In [4]:
# ============================================================================
# FAST VERSION - STAGES 3 & 4 (Should complete in <1 minute)
# ============================================================================

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from pathlib import Path
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

print("\n" + "="*80)
print("FAST DATASET & DATALOADER CREATION")
print("="*80 + "\n")

# ===== 1. Price Scaler =====
price_scaler = StandardScaler()
price_scaler.fit(df[['price(USD)']])
print("✓ Price scaler created")

# ===== 2. Simple Dataset Class (NO fancy features) =====
class FastHousePriceDataset(Dataset):
    def __init__(self, df, image_dir, transform=None, price_scaler=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = Path(image_dir)
        self.transform = transform
        self.price_scaler = price_scaler
        
        # Simple features
        numeric_cols = ['building_area(m²)', 'land_area(m²)', 'bedrooms', 'bathrooms']
        location_encoder = LabelEncoder()
        df['location_encoded'] = location_encoder.fit_transform(df['location'].fillna('Unknown'))
        
        scaler = StandardScaler()
        self.features = scaler.fit_transform(df[numeric_cols + ['location_encoded']].fillna(0))
        self.prices = torch.FloatTensor(df['price(USD)'].values)
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Load image
        img_path = self.image_dir / f"{row['id']}.jpg"
        try:
            image = Image.open(img_path).convert('RGB')
        except:
            image = Image.new('RGB', (224, 224), color=(128, 128, 128))
        
        if self.transform:
            image = self.transform(image)
        
        features = torch.FloatTensor(self.features[idx])
        
        price = float(row['price(USD)'])
        if self.price_scaler:
            price = self.price_scaler.transform([[price]])[0][0]
        
        return image, features, torch.tensor(price, dtype=torch.float32)

print("✓ Dataset class defined")

# ===== 3. Simple Transforms =====
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("✓ Transforms defined")

# ===== 4. Split Data =====
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"✓ Data split: {len(train_df)} train, {len(val_df)} val, {len(test_df)} test")

# ===== 5. Create Datasets =====
train_dataset = FastHousePriceDataset(train_df, image_dir, train_transform, price_scaler)
val_dataset = FastHousePriceDataset(val_df, image_dir, val_transform, price_scaler)
test_dataset = FastHousePriceDataset(test_df, image_dir, val_transform, price_scaler)

print(f"✓ Datasets created: {len(train_dataset)} / {len(val_dataset)} / {len(test_dataset)}")

# ===== 6. Create DataLoaders (SIMPLE - NO fancy features) =====

# Determine batch size
if torch.cuda.is_available():
    batch_size = 16
    num_workers = 0  # ← KEY FIX: Set to 0 for Windows/debugging
else:
    batch_size = 4
    num_workers = 0

print(f"\n⚙️  Settings: batch_size={batch_size}, num_workers={num_workers}")

# Create loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,  # ← Simple shuffle, no weighted sampler
    num_workers=num_workers,
    pin_memory=torch.cuda.is_available(),
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size * 2,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=torch.cuda.is_available()
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size * 2,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=torch.cuda.is_available()
)

print(f"✓ DataLoaders created")
print(f"   Train: {len(train_loader)} batches")
print(f"   Val: {len(val_loader)} batches")
print(f"   Test: {len(test_loader)} batches")

# ===== 7. MINIMAL Test (should be instant) =====
print("\n🧪 Quick test...")
try:
    batch = next(iter(train_loader))
    images, features, prices = batch
    print(f"✅ Working! Shapes: {images.shape}, {features.shape}, {prices.shape}")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n" + "="*80)
print("✅ COMPLETE - Ready for training!")
print("="*80 + "\n")


FAST DATASET & DATALOADER CREATION

✓ Price scaler created
✓ Dataset class defined
✓ Transforms defined
✓ Data split: 1129 train, 242 val, 242 test
✓ Datasets created: 1129 / 242 / 242

⚙️  Settings: batch_size=16, num_workers=0
✓ DataLoaders created
   Train: 70 batches
   Val: 8 batches
   Test: 8 batches

🧪 Quick test...
✅ Working! Shapes: torch.Size([16, 3, 224, 224]), torch.Size([16, 5]), torch.Size([16])

✅ COMPLETE - Ready for training!



Stage 3: Data Preparation and Split

In [6]:
# ============================================================================
# STAGE 3: DATA PREPARATION AND SPLIT (COMPLETE VERSION)
# ============================================================================

# Create price scaler for normalization
price_scaler = StandardScaler()
price_scaler.fit(df[['price(USD)']])
print(f'✓ Price scaler created for normalization')

# Dataset class
class HousePriceDataset(Dataset):
    def __init__(self, df, image_dir, transform=None, include_features=True, price_scaler=None):
        self.df = df.reset_index(drop=True).copy()
        self.image_dir = Path(image_dir)
        self.transform = transform
        self.include_features = include_features
        self.price_scaler = price_scaler
        
        # Prepare numeric features
        self.numeric_features = ['building_area(m²)', 'land_area(m²)', 'bedrooms', 'bathrooms']
        
        # Prepare location encoding
        self.location_encoder = LabelEncoder()
        self.df['location_encoded'] = self.location_encoder.fit_transform(df['location'].fillna('Unknown'))
        
        # Normalize numeric features
        self.scaler = StandardScaler()
        feature_cols = self.numeric_features + ['location_encoded']
        self.feature_array = self.scaler.fit_transform(self.df[feature_cols].fillna(0))
        
        # Store prices for sampling
        self.prices = df['price(USD)'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Load image
        img_path = self.image_dir / f"{row['id']}.jpg"
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            # If image fails to load, create a black image
            image = Image.new('RGB', (224, 224), color='black')
        
        if self.transform:
            image = self.transform(image)
        
        # Get features (ensure Float32)
        features = torch.FloatTensor(self.feature_array[idx])
        
        # Get target (price) - normalize if scaler is provided
        raw_price = float(row['price(USD)'])
        if self.price_scaler is not None:
            # Normalize the price
            price_normalized = self.price_scaler.transform([[raw_price]])[0][0]
            price = torch.tensor(price_normalized, dtype=torch.float32)
        else:
            price = torch.tensor(raw_price, dtype=torch.float32)
        
        if self.include_features:
            return image, features, price
        else:
            return image, price

# Data transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Split data
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f'Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}')

# ============================================================================
# ACTUALLY CREATE THE DATASETS (THIS WAS MISSING!)
# ============================================================================

# Create datasets with price scaler
train_dataset = HousePriceDataset(train_df, image_dir, transform=train_transform, price_scaler=price_scaler)
val_dataset = HousePriceDataset(val_df, image_dir, transform=val_transform, price_scaler=price_scaler)
test_dataset = HousePriceDataset(test_df, image_dir, transform=val_transform, price_scaler=price_scaler)

print('✅ Datasets created:')
print(f'   Train dataset: {len(train_dataset)} samples')
print(f'   Val dataset: {len(val_dataset)} samples')  
print(f'   Test dataset: {len(test_dataset)} samples')

# Quick test
sample = train_dataset[0]
print(f'✅ Sample shapes - Images: {sample[0].shape}, Features: {sample[1].shape}, Price: {sample[2].shape}')

✓ Price scaler created for normalization
Train: 1129, Val: 242, Test: 242
✅ Datasets created:
   Train dataset: 1129 samples
   Val dataset: 242 samples
   Test dataset: 242 samples
✅ Sample shapes - Images: torch.Size([3, 224, 224]), Features: torch.Size([5]), Price: torch.Size([])


Stage 4: Create DataLoaders

In [ ]:
# ============================================================================
# STAGE 4: SIMPLIFIED DATALOADERS (WORKS WITH STAGE 3)
# ============================================================================

import torch
from torch.utils.data import DataLoader
import os

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

def create_simple_dataloaders(train_dataset, val_dataset, test_dataset):
    """
    Create simple but optimized dataloaders without complex dependencies
    """
    print(f'\n{"="*80}')
    print(f'📦 CREATING DATALOADERS')
    print(f'{"="*80}')
    
    # Determine optimal settings
    if torch.cuda.is_available():
        batch_size = 16  # Good for RTX 4060
        num_workers = min(4, os.cpu_count() - 1) if os.cpu_count() else 2
        if os.name == 'nt':  # Windows
            num_workers = min(2, num_workers)
    else:
        batch_size = 8
        num_workers = 2
    
    print(f'⚙️  Settings:')
    print(f'   Batch size: {batch_size}')
    print(f'   Workers: {num_workers}')
    print(f'   Device: {device}')
    
    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,  # Simple shuffle instead of weighted sampler
        num_workers=num_workers,
        pin_memory=torch.cuda.is_available(),
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=torch.cuda.is_available(),
        drop_last=False
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=torch.cuda.is_available(),
        drop_last=False
    )
    
    print(f'\n✅ Data loaders created:')
    print(f'   Train batches: {len(train_loader)}')
    print(f'   Val batches: {len(val_loader)}')
    print(f'   Test batches: {len(test_loader)}')
    
    # Test data loading
    print(f'\n🧪 Testing data loading...')
    try:
        test_batch = next(iter(train_loader))
        images, features, prices = test_batch
        print(f'✅ Data loading works!')
        print(f'   Batch shape - Images: {images.shape}, Features: {features.shape}, Prices: {prices.shape}')
        print(f'   Price range: {prices.min().item():.4f} to {prices.max().item():.4f}')
    except Exception as e:
        print(f'❌ Data loading error: {e}')
        raise
    
    print(f'{"="*80}\n')
    
    return train_loader, val_loader, test_loader

# ============================================================================
# ACTUALLY CREATE THE DATALOADERS (THIS WAS MISSING!)
# ============================================================================

# Create dataloaders using the datasets from Stage 3
train_loader, val_loader, test_loader = create_simple_dataloaders(
    train_dataset=train_dataset,
    val_dataset=val_dataset, 
    test_dataset=test_dataset
)

print('🎉 Stage 4 complete - DataLoaders ready for training!')

Using device: cuda

📦 CREATING DATALOADERS
⚙️  Settings:
   Batch size: 16
   Workers: 2
   Device: cuda

✅ Data loaders created:
   Train batches: 70
   Val batches: 16
   Test batches: 16

🧪 Testing data loading...


Stage 5: ULTRA-STABLE Model Architecture

In [6]:
# ============================================================================
# COMPLETE WORKING STAGE 5 - ULTRA-STABLE MODEL
# ============================================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from typing import Tuple
import math

print("\n" + "="*80)
print("STAGE 5: ULTRA-STABLE MODEL ARCHITECTURE")
print("="*80 + "\n")

# ============================================================================
# STABILITY UTILITIES
# ============================================================================

class SafeBatchNorm1d(nn.BatchNorm1d):
    """BatchNorm that prevents NaN/Inf"""
    def forward(self, x):
        x = torch.clamp(x, min=-1e4, max=1e4)
        output = super().forward(x)
        if not torch.isfinite(output).all():
            return torch.nan_to_num(output, nan=0.0, posinf=1.0, neginf=-1.0)
        return output

class GradientClipLinear(nn.Linear):
    """Linear layer with built-in gradient clipping"""
    def __init__(self, *args, max_grad_norm=1.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.max_grad_norm = max_grad_norm
        self.weight.register_hook(
            lambda grad: torch.clamp(grad, -max_grad_norm, max_grad_norm)
        )
        if self.bias is not None:
            self.bias.register_hook(
                lambda grad: torch.clamp(grad, -max_grad_norm, max_grad_norm)
            )

# ============================================================================
# PREDICTION HEAD (FIXED)
# ============================================================================

class UltraStablePredictionHead(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dims: Tuple[int, ...] = (512, 256, 128, 64),
        dropout: float = 0.3
    ):
        super().__init__()
        
        layers = []
        prev_dim = input_dim
        
        for i, hidden_dim in enumerate(hidden_dims):
            layers.append(GradientClipLinear(prev_dim, hidden_dim))
            layers.append(SafeBatchNorm1d(hidden_dim))
            layers.append(nn.ReLU(inplace=False))
            dropout_rate = dropout * (1.0 - 0.2 * i / len(hidden_dims))
            layers.append(nn.Dropout(dropout_rate))
            prev_dim = hidden_dim
        
        self.feature_extractor = nn.Sequential(*layers)  # ← FIX: Use *layers
        
        self.output_layer = nn.Linear(hidden_dims[-1], 1)
        nn.init.normal_(self.output_layer.weight, mean=0.0, std=0.001)
        nn.init.constant_(self.output_layer.bias, 0.0)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        features = self.feature_extractor(x)
        output = self.output_layer(features)
        output = torch.clamp(output, min=-15.0, max=15.0)
        return output.squeeze(-1)

# ============================================================================
# MAIN MODEL
# ============================================================================

class UltraStableHousePriceModel(nn.Module):
    def __init__(
        self,
        backbone_name: str = 'efficientnet_b0',
        num_tabular_features: int = 5,
        dropout: float = 0.3,
        pretrained: bool = True
    ):
        super().__init__()
        
        self.backbone_name = backbone_name
        
        # Create backbone
        self.backbone, self.image_feature_dim = self._create_backbone(
            backbone_name, pretrained
        )
        
        # Tabular encoder
        self.tabular_encoder = nn.Sequential(
            GradientClipLinear(num_tabular_features, 64, max_grad_norm=1.0),
            SafeBatchNorm1d(64),
            nn.ReLU(inplace=False),
            nn.Dropout(0.2),
            GradientClipLinear(64, 128, max_grad_norm=1.0),
            SafeBatchNorm1d(128),
            nn.ReLU(inplace=False),
            nn.Dropout(0.2),
        )
        self.tabular_feature_dim = 128
        
        # Fusion (simple concatenation)
        fusion_dim = self.image_feature_dim + self.tabular_feature_dim
        
        # Prediction head
        self.prediction_head = UltraStablePredictionHead(
            input_dim=fusion_dim,
            hidden_dims=(512, 256, 128, 64),
            dropout=dropout
        )
        
        self._initialize_weights()
        
        print(f"✓ Model created: {backbone_name}")
        print(f"  Image features: {self.image_feature_dim}")
        print(f"  Tabular features: {self.tabular_feature_dim}")
        print(f"  Fusion dim: {fusion_dim}")
        print(f"  Total parameters: {sum(p.numel() for p in self.parameters()):,}")
    
    def _create_backbone(self, backbone_name: str, pretrained: bool) -> Tuple[nn.Module, int]:
        backbone_name = backbone_name.lower()
        
        if 'efficientnet_b0' in backbone_name:
            backbone = models.efficientnet_b0(
                weights=models.EfficientNet_B0_Weights.DEFAULT if pretrained else None
            )
            feature_dim = 1280
            backbone.classifier = nn.Identity()
            
        elif 'efficientnet_b3' in backbone_name:
            backbone = models.efficientnet_b3(
                weights=models.EfficientNet_B3_Weights.DEFAULT if pretrained else None
            )
            feature_dim = 1536
            backbone.classifier = nn.Identity()
            
        elif 'resnet50' in backbone_name:
            backbone = models.resnet50(
                weights=models.ResNet50_Weights.DEFAULT if pretrained else None
            )
            feature_dim = 2048
            backbone.fc = nn.Identity()
            
        elif 'mobilenet_v2' in backbone_name:
            backbone = models.mobilenet_v2(
                weights=models.MobileNet_V2_Weights.DEFAULT if pretrained else None
            )
            feature_dim = 1280
            backbone.classifier = nn.Identity()
            
        elif 'densenet121' in backbone_name:
            backbone = models.densenet121(
                weights=models.DenseNet121_Weights.DEFAULT if pretrained else None
            )
            feature_dim = 1024
            backbone.classifier = nn.Identity()
        
        else:
            print(f"⚠️  Unknown backbone '{backbone_name}', using EfficientNet-B0")
            backbone = models.efficientnet_b0(
                weights=models.EfficientNet_B0_Weights.DEFAULT if pretrained else None
            )
            feature_dim = 1280
            backbone.classifier = nn.Identity()
        
        return backbone, feature_dim
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight, gain=0.5)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, images: torch.Tensor, tabular_features: torch.Tensor) -> torch.Tensor:
        # Extract image features
        if not torch.isfinite(images).all():
            images = torch.nan_to_num(images, nan=0.0, posinf=1.0, neginf=0.0)
            images = torch.clamp(images, min=0.0, max=1.0)
        
        image_features = self.backbone(images)
        if len(image_features.shape) > 2:
            image_features = torch.flatten(image_features, 1)
        
        if not torch.isfinite(image_features).all():
            image_features = torch.nan_to_num(image_features, nan=0.0, posinf=1.0, neginf=-1.0)
        
        # Process tabular features
        if not torch.isfinite(tabular_features).all():
            tabular_features = torch.nan_to_num(tabular_features, nan=0.0, posinf=1.0, neginf=-1.0)
        
        tabular_encoded = self.tabular_encoder(tabular_features)
        
        if not torch.isfinite(tabular_encoded).all():
            tabular_encoded = torch.nan_to_num(tabular_encoded, nan=0.0, posinf=1.0, neginf=-1.0)
        
        # Fuse features (concatenation)
        fused_features = torch.cat([image_features, tabular_encoded], dim=1)
        
        if not torch.isfinite(fused_features).all():
            fused_features = torch.nan_to_num(fused_features, nan=0.0, posinf=1.0, neginf=-1.0)
        
        # Predict price
        predictions = self.prediction_head(fused_features)
        
        if not torch.isfinite(predictions).all():
            predictions = torch.nan_to_num(predictions, nan=0.0, posinf=10.0, neginf=-10.0)
            predictions = torch.clamp(predictions, min=-10.0, max=15.0)
        
        return predictions

# ============================================================================
# TEST THE MODEL
# ============================================================================

print("\n🧪 Testing model creation...")

# Create model
model = UltraStableHousePriceModel(
    backbone_name='efficientnet_b0',
    num_tabular_features=5,
    dropout=0.3,
    pretrained=True
)

model = model.to(device)
print(f"\n✅ Model moved to {device}")

# Test forward pass
print("\n🧪 Testing forward pass...")
dummy_images = torch.randn(2, 3, 224, 224).to(device)
dummy_features = torch.randn(2, 5).to(device)

model.eval()
with torch.no_grad():
    output = model(dummy_images, dummy_features)

print(f"✅ Forward pass successful!")
print(f"   Input: {dummy_images.shape}, {dummy_features.shape}")
print(f"   Output: {output.shape}")
print(f"   Output range: [{output.min():.3f}, {output.max():.3f}]")

assert torch.isfinite(output).all(), "❌ NaN/Inf in output!"
print(f"✅ No NaN/Inf detected")

print("\n" + "="*80)
print("✅ STAGE 5 COMPLETE - MODEL READY")
print("="*80 + "\n")

print("📊 Model Summary:")
print(f"   Architecture: {model.backbone_name}")
print(f"   Image features: {model.image_feature_dim}")
print(f"   Tabular features: {model.tabular_feature_dim}")
print(f"   Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"   Trainable: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print("\n🚀 Ready for training!")


STAGE 5: ULTRA-STABLE MODEL ARCHITECTURE


🧪 Testing model creation...
✓ Model created: efficientnet_b0
  Image features: 1280
  Tabular features: 128
  Fusion dim: 1408
  Total parameters: 4,912,509

✅ Model moved to cuda

🧪 Testing forward pass...
✅ Forward pass successful!
   Input: torch.Size([2, 3, 224, 224]), torch.Size([2, 5])
   Output: torch.Size([2])
   Output range: [0.000, 0.000]
✅ No NaN/Inf detected

✅ STAGE 5 COMPLETE - MODEL READY

📊 Model Summary:
   Architecture: efficientnet_b0
   Image features: 1280
   Tabular features: 128
   Total parameters: 4,912,509
   Trainable: 4,912,509

🚀 Ready for training!


Stage 6: ULTRA-STABLE Training Function with Gradient Safety

In [ ]:

    
# ============================================================================
# STAGE 6: ULTRA-STABLE TRAINING FUNCTION
# ============================================================================

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, OneCycleLR, ReduceLROnPlateau
from torch.cuda.amp import autocast, GradScaler
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tqdm import tqdm
import time
import copy
from collections import defaultdict
import os
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# ULTRA-STABLE LOSS FUNCTIONS
# ============================================================================

class SafeHuberLoss(nn.Module):
    """Huber loss with NaN/Inf protection"""
    def __init__(self, delta=1.0):
        super().__init__()
        self.delta = delta
    
    def forward(self, pred, target):
        # Clamp inputs to prevent extreme values
        pred = torch.clamp(pred, min=-20, max=20)
        target = torch.clamp(target, min=-20, max=20)
        
        error = torch.abs(pred - target)
        quadratic = torch.min(error, torch.tensor(self.delta, device=error.device))
        linear = error - quadratic
        loss = torch.mean(0.5 * quadratic**2 + self.delta * linear)
        
        # Safety check
        if not torch.isfinite(loss):
            return torch.tensor(1.0, device=pred.device, requires_grad=True)
        
        return loss


class SafeCombinedLoss(nn.Module):
    """Combined loss with stability guarantees"""
    def __init__(self, alpha=0.4, beta=0.3, gamma=0.3):
        super().__init__()
        self.alpha = alpha  # MSE weight
        self.beta = beta    # MAE weight  
        self.gamma = gamma  # Huber weight
        self.huber = SafeHuberLoss(delta=1.0)
    
    def forward(self, pred, target):
        # Clamp to prevent extreme values
        pred = torch.clamp(pred, min=-20, max=20)
        target = torch.clamp(target, min=-20, max=20)
        
        # MSE component
        mse = torch.mean((pred - target) ** 2)
        
        # MAE component
        mae = torch.mean(torch.abs(pred - target))
        
        # Huber component
        huber = self.huber(pred, target)
        
        # Combined loss
        loss = self.alpha * mse + self.beta * mae + self.gamma * huber
        
        # Final safety check
        if not torch.isfinite(loss):
            return torch.tensor(1.0, device=pred.device, requires_grad=True)
        
        return loss


# ============================================================================
# GRADIENT SAFETY UTILITIES
# ============================================================================

def check_model_gradients(model, max_norm=10.0):
    """
    Check model gradients for NaN/Inf and extreme values
    Returns: (has_nan, has_inf, max_grad_norm)
    """
    has_nan = False
    has_inf = False
    max_grad = 0.0
    
    for name, param in model.named_parameters():
        if param.grad is not None:
            grad = param.grad
            
            # Check for NaN
            if torch.isnan(grad).any():
                has_nan = True
                logger.warning(f"⚠️  NaN gradient in {name}")
            
            # Check for Inf
            if torch.isinf(grad).any():
                has_inf = True
                logger.warning(f"⚠️  Inf gradient in {name}")
            
            # Check magnitude
            grad_norm = grad.norm().item()
            max_grad = max(max_grad, grad_norm)
            
            if grad_norm > max_norm:
                logger.warning(f"⚠️  Large gradient ({grad_norm:.2f}) in {name}")
    
    return has_nan, has_inf, max_grad


def sanitize_gradients(model):
    """Replace NaN/Inf gradients with zeros"""
    for name, param in model.named_parameters():
        if param.grad is not None:
            param.grad = torch.nan_to_num(
                param.grad, 
                nan=0.0, 
                posinf=0.0, 
                neginf=0.0
            )


def check_model_weights(model):
    """Check model weights for NaN/Inf"""
    for name, param in model.named_parameters():
        if not torch.isfinite(param).all():
            logger.error(f"❌ NaN/Inf in weights: {name}")
            return False
    return True


# ============================================================================
# ULTRA-STABLE TRAINING FUNCTION
# ============================================================================

def train_ultra_stable(
    model,
    train_loader,
    val_loader,
    test_loader=None,
    num_epochs=50,
    lr=0.001,  # LOWER default learning rate
    loss_type='combined',
    optimizer_type='adamw',
    scheduler_type='plateau',
    gradient_clip_norm=0.5,  # AGGRESSIVE clipping
    gradient_accumulation_steps=1,
    warmup_epochs=5,  # Longer warmup
    early_stopping_patience=15,
    save_best_model=True,
    checkpoint_dir='./checkpoints',
    model_name='model',
    device='cuda'
):
    """
    Ultra-stable training with maximum gradient safety
    
    Key Safety Features:
    1. Aggressive gradient clipping (0.5 default)
    2. NaN/Inf detection at every step
    3. Automatic gradient sanitization
    4. Weight validation after each epoch
    5. Safe loss clamping
    6. Conservative learning rates
    7. Extended warmup period
    8. Automatic recovery from instability
    """
    
    print(f'\n{"="*80}')
    print(f'🛡️  ULTRA-STABLE TRAINING: {model_name}')
    print(f'{"="*80}')
    
    # ========================================================================
    # SETUP
    # ========================================================================
    
    # Loss function with stability
    print(f'\n🎯 Loss Function: {loss_type.upper()}')
    if loss_type == 'combined':
        criterion = SafeCombinedLoss(alpha=0.4, beta=0.3, gamma=0.3)
    elif loss_type == 'huber':
        criterion = SafeHuberLoss(delta=1.0)
    elif loss_type == 'mse':
        criterion = nn.MSELoss()
    else:
        criterion = SafeCombinedLoss()
    
    # Optimizer with conservative settings
    print(f'⚙️  Optimizer: {optimizer_type.upper()}')
    if optimizer_type == 'adamw':
        optimizer = optim.AdamW(
            model.parameters(),
            lr=lr,
            weight_decay=1e-4,
            betas=(0.9, 0.999),
            eps=1e-8  # Stability
        )
    elif optimizer_type == 'adam':
        optimizer = optim.Adam(
            model.parameters(),
            lr=lr,
            weight_decay=1e-4,
            eps=1e-8
        )
    elif optimizer_type == 'sgd':
        optimizer = optim.SGD(
            model.parameters(),
            lr=lr,
            momentum=0.9,
            weight_decay=1e-4,
            nesterov=True
        )
    else:
        optimizer = optim.AdamW(model.parameters(), lr=lr)
    
    # Learning rate scheduler
    print(f'📈 Scheduler: {scheduler_type.upper()}')
    if scheduler_type == 'plateau':
        scheduler = ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.5,
            patience=7,
            min_lr=1e-7,
            cooldown=3,
            verbose=True
        )
    elif scheduler_type == 'cosine':
        scheduler = CosineAnnealingWarmRestarts(
            optimizer,
            T_0=10,
            T_mult=2,
            eta_min=1e-7
        )
    elif scheduler_type == 'onecycle':
        total_steps = len(train_loader) * num_epochs
        scheduler = OneCycleLR(
            optimizer,
            max_lr=lr * 5,  # Conservative multiplier
            total_steps=total_steps,
            pct_start=0.3,
            anneal_strategy='cos'
        )
    else:
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=7)
    
    # Mixed precision with safety
    use_amp = torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 7
    scaler = GradScaler(enabled=use_amp)
    if use_amp:
        print(f'⚡ Mixed Precision: ENABLED')
    
    # Training configuration
    print(f'\n📋 Training Configuration:')
    print(f'   Epochs: {num_epochs}')
    print(f'   Learning Rate: {lr}')
    print(f'   Gradient Clip: {gradient_clip_norm}')
    print(f'   Warmup Epochs: {warmup_epochs}')
    print(f'   Accumulation Steps: {gradient_accumulation_steps}')
    print(f'   Early Stopping Patience: {early_stopping_patience}')
    print(f'   Device: {device}')
    
    # Training state
    history = defaultdict(list)
    best_val_loss = float('inf')
    best_val_r2 = -float('inf')
    best_model_state = None
    no_improve_count = 0
    nan_inf_count = 0
    max_nan_inf_tolerance = 3  # Stop if too many NaN/Inf episodes
    
    start_time = time.time()
    
    # Create checkpoint directory
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # ========================================================================
    # MAIN TRAINING LOOP
    # ========================================================================
    
    for epoch in range(num_epochs):
        epoch_start = time.time()
        
        # Check model weights before epoch
        if not check_model_weights(model):
            logger.error("❌ Model weights corrupted! Stopping training.")
            break
        
        # ====================================================================
        # WARMUP LEARNING RATE
        # ====================================================================
        if epoch < warmup_epochs:
            warmup_factor = (epoch + 1) / warmup_epochs
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr * warmup_factor
            logger.info(f"Warmup LR: {lr * warmup_factor:.2e}")
        
        # ====================================================================
        # TRAINING PHASE
        # ====================================================================
        model.train()
        train_loss = 0.0
        train_preds_list = []
        train_targets_list = []
        batch_nan_count = 0
        
        train_pbar = tqdm(
            train_loader,
            desc=f'Epoch {epoch+1}/{num_epochs} [Train]',
            ncols=120
        )
        
        optimizer.zero_grad()
        
        for batch_idx, (images, features, prices) in enumerate(train_pbar):
            try:
                # Move to device
                images = images.to(device, non_blocking=True)
                features = features.to(device, non_blocking=True)
                prices = prices.to(device, non_blocking=True)
                
                # Safety check on inputs
                if not torch.isfinite(images).all():
                    logger.warning(f"⚠️  NaN/Inf in images batch {batch_idx}")
                    images = torch.nan_to_num(images, nan=0.0, posinf=1.0, neginf=0.0)
                
                if not torch.isfinite(features).all():
                    logger.warning(f"⚠️  NaN/Inf in features batch {batch_idx}")
                    features = torch.nan_to_num(features, nan=0.0, posinf=1.0, neginf=-1.0)
                
                # Forward pass with mixed precision
                with autocast(enabled=use_amp):
                    outputs = model(images, features)
                    
                    # Check outputs
                    if not torch.isfinite(outputs).all():
                        logger.warning(f"⚠️  NaN/Inf in model outputs batch {batch_idx}!")
                        outputs = torch.nan_to_num(outputs, nan=0.0, posinf=10.0, neginf=-10.0)
                        outputs = torch.clamp(outputs, min=-15.0, max=15.0)
                        batch_nan_count += 1
                    
                    # Compute loss
                    loss = criterion(outputs, prices)
                    loss = loss / gradient_accumulation_steps
                    
                    # Check loss
                    if not torch.isfinite(loss):
                        logger.warning(f"⚠️  NaN/Inf loss batch {batch_idx}! Skipping...")
                        batch_nan_count += 1
                        continue
                
                # Backward pass
                scaler.scale(loss).backward()
                
                # Gradient accumulation
                if (batch_idx + 1) % gradient_accumulation_steps == 0:
                    # Check gradients
                    has_nan, has_inf, max_grad = check_model_gradients(model)
                    
                    if has_nan or has_inf:
                        logger.warning(f"⚠️  NaN/Inf gradients detected! Sanitizing...")
                        sanitize_gradients(model)
                        batch_nan_count += 1
                    
                    # Unscale gradients
                    scaler.unscale_(optimizer)
                    
                    # AGGRESSIVE gradient clipping
                    torch.nn.utils.clip_grad_norm_(
                        model.parameters(),
                        max_norm=gradient_clip_norm
                    )
                    
                    # Optimizer step
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
                    
                    # Update OneCycle scheduler
                    if scheduler_type == 'onecycle':
                        scheduler.step()
                
                # Accumulate loss and predictions
                train_loss += loss.item() * gradient_accumulation_steps
                train_preds_list.extend(outputs.detach().cpu().numpy())
                train_targets_list.extend(prices.cpu().numpy())
                
                # Update progress bar
                if (batch_idx + 1) % 5 == 0:
                    current_lr = optimizer.param_groups[0]['lr']
                    train_pbar.set_postfix({
                        'loss': f'{loss.item() * gradient_accumulation_steps:.4f}',
                        'lr': f'{current_lr:.2e}',
                        'nan': batch_nan_count
                    })
                
            except RuntimeError as e:
                logger.error(f"❌ Runtime error in batch {batch_idx}: {e}")
                if "out of memory" in str(e):
                    logger.error("💥 GPU OOM! Clearing cache...")
                    torch.cuda.empty_cache()
                continue
        
        # Check for excessive NaN/Inf in epoch
        if batch_nan_count > len(train_loader) * 0.1:  # >10% batches corrupted
            logger.error(f"❌ Too many NaN/Inf batches ({batch_nan_count})! Epoch unstable.")
            nan_inf_count += 1
            if nan_inf_count >= max_nan_inf_tolerance:
                logger.error("💥 Training unstable! Stopping...")
                break
        
        avg_train_loss = train_loss / len(train_loader)
        
        # ====================================================================
        # CALCULATE TRAINING METRICS (SAFELY)
        # ====================================================================
        train_preds_norm = np.array(train_preds_list).reshape(-1, 1)
        train_targets_norm = np.array(train_targets_list).reshape(-1, 1)
        
        # Safety checks
        if not np.isfinite(train_preds_norm).all():
            logger.warning('⚠️  NaN/Inf in training predictions!')
            train_preds_norm = np.nan_to_num(train_preds_norm, nan=0.0, posinf=5.0, neginf=-5.0)
            train_preds_norm = np.clip(train_preds_norm, -15, 15)
        
        try:
            # Denormalize
            train_preds_denorm = train_loader.dataset.price_scaler.inverse_transform(
                train_preds_norm
            ).flatten()
            train_targets_denorm = train_loader.dataset.price_scaler.inverse_transform(
                train_targets_norm
            ).flatten()
            
            # Calculate metrics
            train_r2 = r2_score(train_targets_denorm, train_preds_denorm)
            train_rmse = np.sqrt(mean_squared_error(train_targets_denorm, train_preds_denorm))
            train_mae = mean_absolute_error(train_targets_denorm, train_preds_denorm)
            
            # Clamp unrealistic metrics
            train_r2 = max(-10.0, min(1.0, train_r2))
            
        except Exception as e:
            logger.error(f"❌ Error calculating training metrics: {e}")
            train_r2 = 0.0
            train_rmse = float('inf')
            train_mae = float('inf')
        
        # ====================================================================
        # VALIDATION PHASE
        # ====================================================================
        model.eval()
        val_loss = 0.0
        val_preds_list = []
        val_targets_list = []
        
        val_pbar = tqdm(
            val_loader,
            desc=f'Epoch {epoch+1}/{num_epochs} [Val]',
            leave=False,
            ncols=120
        )
        
        with torch.no_grad():
            for images, features, prices in val_pbar:
                try:
                    images = images.to(device, non_blocking=True)
                    features = features.to(device, non_blocking=True)
                    prices = prices.to(device, non_blocking=True)
                    
                    # Forward pass
                    with autocast(enabled=use_amp):
                        outputs = model(images, features)
                        
                        # Safety check
                        if not torch.isfinite(outputs).all():
                            outputs = torch.nan_to_num(outputs, nan=0.0, posinf=10.0, neginf=-10.0)
                            outputs = torch.clamp(outputs, min=-15.0, max=15.0)
                        
                        loss = criterion(outputs, prices)
                        
                        if not torch.isfinite(loss):
                            logger.warning("⚠️  NaN/Inf in validation loss!")
                            continue
                    
                    val_loss += loss.item()
                    val_preds_list.extend(outputs.cpu().numpy())
                    val_targets_list.extend(prices.cpu().numpy())
                    
                except Exception as e:
                    logger.error(f"❌ Validation error: {e}")
                    continue
        
        avg_val_loss = val_loss / len(val_loader) if len(val_loader) > 0 else float('inf')
        
        # ====================================================================
        # CALCULATE VALIDATION METRICS (SAFELY)
        # ====================================================================
        val_preds_norm = np.array(val_preds_list).reshape(-1, 1)
        val_targets_norm = np.array(val_targets_list).reshape(-1, 1)
        
        if not np.isfinite(val_preds_norm).all():
            logger.warning('⚠️  NaN/Inf in validation predictions!')
            val_preds_norm = np.nan_to_num(val_preds_norm, nan=0.0, posinf=5.0, neginf=-5.0)
            val_preds_norm = np.clip(val_preds_norm, -15, 15)
        
        try:
            val_preds_denorm = val_loader.dataset.price_scaler.inverse_transform(
                val_preds_norm
            ).flatten()
            val_targets_denorm = val_loader.dataset.price_scaler.inverse_transform(
                val_targets_norm
            ).flatten()
            
            val_r2 = r2_score(val_targets_denorm, val_preds_denorm)
            val_rmse = np.sqrt(mean_squared_error(val_targets_denorm, val_preds_denorm))
            val_mae = mean_absolute_error(val_targets_denorm, val_preds_denorm)
            
            # Clamp metrics
            val_r2 = max(-10.0, min(1.0, val_r2))
            
        except Exception as e:
            logger.error(f"❌ Error calculating validation metrics: {e}")
            val_r2 = 0.0
            val_rmse = float('inf')
            val_mae = float('inf')
        
        # ====================================================================
        # UPDATE SCHEDULER
        # ====================================================================
        if scheduler_type == 'plateau':
            scheduler.step(avg_val_loss)
        elif scheduler_type == 'cosine':
            scheduler.step()
        
        # ====================================================================
        # SAVE HISTORY
        # ====================================================================
        history['train_loss'].append(avg_train_loss)
        history['train_r2'].append(train_r2)
        history['train_rmse'].append(train_rmse)
        history['train_mae'].append(train_mae)
        
        history['val_loss'].append(avg_val_loss)
        history['val_r2'].append(val_r2)
        history['val_rmse'].append(val_rmse)
        history['val_mae'].append(val_mae)
        history['lr'].append(optimizer.param_groups[0]['lr'])
        
        epoch_time = time.time() - epoch_start
        history['epoch_time'].append(epoch_time)
        
        # ====================================================================
        # LOGGING
        # ====================================================================
        current_lr = optimizer.param_groups[0]['lr']
        is_best = (avg_val_loss < best_val_loss and np.isfinite(avg_val_loss))
        
        print(f'\n{"="*80}')
        print(f'Epoch {epoch+1}/{num_epochs} Summary ({epoch_time:.2f}s)')
        print(f'{"="*80}')
        print(f'📉 Train Loss: {avg_train_loss:.6f} | R²: {train_r2:.4f} | RMSE: ${train_rmse:,.0f}')
        print(f'📊 Val Loss: {avg_val_loss:.6f} | R²: {val_r2:.4f} | RMSE: ${val_rmse:,.0f}')
        print(f'🎯 Val MAE: ${val_mae:,.0f}')
        print(f'🔧 Learning Rate: {current_lr:.2e}')
        print(f'⚠️  NaN/Inf Batches: {batch_nan_count}')
        
        if is_best:
            print(f'⭐ NEW BEST MODEL!')
        
        # ====================================================================
        # MODEL CHECKPOINTING
        # ====================================================================
        if is_best:
            best_val_loss = avg_val_loss
            best_val_r2 = val_r2
            best_model_state = copy.deepcopy(model.state_dict())
            no_improve_count = 0
            
            if save_best_model:
                checkpoint_path = os.path.join(checkpoint_dir, f'{model_name}_best.pth')
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'val_loss': best_val_loss,
                    'val_r2': best_val_r2,
                    'history': dict(history)
                }, checkpoint_path)
                print(f'💾 Checkpoint saved: {checkpoint_path}')
        else:
            no_improve_count += 1
            print(f'⏳ No improvement for {no_improve_count}/{early_stopping_patience} epochs')
            
            if no_improve_count >= early_stopping_patience:
                print(f'\n🛑 Early stopping at epoch {epoch+1}')
                break
    
    # ========================================================================
    # LOAD BEST MODEL
    # ========================================================================
    if best_model_state:
        model.load_state_dict(best_model_state)
        print('\n✅ Best model loaded')
    
    training_time = time.time() - start_time
    
    # ========================================================================
    # FINAL TEST EVALUATION
    # ========================================================================
    if test_loader is not None:
        print('\n' + '='*80)
        print('🧪 FINAL TEST EVALUATION')
        print('='*80)
        
        model.eval()
        test_preds_list = []
        test_targets_list = []
        
        with torch.no_grad():
            for images, features, prices in tqdm(test_loader, desc='Testing', ncols=120):
                images = images.to(device, non_blocking=True)
                features = features.to(device, non_blocking=True)
                prices = prices.to(device, non_blocking=True)
                
                outputs = model(images, features)
                
                # Safety
                if not torch.isfinite(outputs).all():
                    outputs = torch.nan_to_num(outputs, nan=0.0, posinf=10.0, neginf=-10.0)
                
                test_preds_list.extend(outputs.cpu().numpy())
                test_targets_list.extend(prices.cpu().numpy())
        
        # Calculate test metrics
        test_preds_norm = np.array(test_preds_list).reshape(-1, 1)
        test_targets_norm = np.array(test_targets_list).reshape(-1, 1)
        
        test_preds = test_loader.dataset.price_scaler.inverse_transform(test_preds_norm).flatten()
        test_targets = test_loader.dataset.price_scaler.inverse_transform(test_targets_norm).flatten()
        
        test_r2 = r2_score(test_targets, test_preds)
        test_rmse = np.sqrt(mean_squared_error(test_targets, test_preds))
        test_mae = mean_absolute_error(test_targets, test_preds)
        
        print(f'\n📊 TEST RESULTS: {model_name}')
        print(f'{"="*80}')
        print(f'  🎯 R² Score:      {test_r2:.4f}')
        print(f'  📉 RMSE:         ${test_rmse:,.0f}')
        print(f'  📏 MAE:          ${test_mae:,.0f}')
        print(f'  ⏱️  Training Time: {training_time/60:.2f} min')
        print(f'{"="*80}')
    else:
        test_r2 = test_rmse = test_mae = None
    
    # ========================================================================
    # COMPILE RESULTS
    # ========================================================================
    results = {
        'model_name': model_name,
        'test_r2': test_r2,
        'test_rmse': test_rmse,
        'test_mae': test_mae,
        'best_val_loss': best_val_loss,
        'best_val_r2': best_val_r2,
        'training_time': training_time,
        'total_epochs': len(history['train_loss']),
        'history': dict(history),
        'nan_inf_episodes': nan_inf_count
    }
    
    return model, results


# ============================================================================
# USAGE EXAMPLE
# ============================================================================

if __name__ == "__main__":
    # Create stable model
    model = create_stable_model(
        backbone_name='efficientnet_b0',  # SAFE BACKBONE
        num_features=5,
        fusion_method='concat',
        dropout=0.3,
        freeze_backbone_epochs=5
    ).to(device)
    
    # Train with ultra-stable settings
    trained_model, results = train_ultra_stable(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        num_epochs=50,
        lr=0.001,  # Conservative LR
        loss_type='combined',
        optimizer_type='adamw',
        scheduler_type='plateau',
        gradient_clip_norm=0.5,  # Aggressive clipping
        warmup_epochs=5,
        early_stopping_patience=15,
        model_name='efficientnet_b0_stable',
        device=device
    )
    
    print(f"\n✅ Training complete!")
    print(f"Final R²: {results['test_r2']:.4f}")
    print(f"Final RMSE: ${results['test_rmse']:,.0f}")

TypeError: UltraStableHousePriceModel.__init__() got an unexpected keyword argument 'fusion_method'

Stage 7: COMPLETE PRE-TRAINING TEST & 21 MODEL ENSEMBLE PIPELINE

In [ ]:
# ============================================================================
# COMPREHENSIVE PRE-TRAINING TEST SUITE
# ============================================================================

import torch
import torch.nn as nn
import numpy as np
from pathlib import Path
import time
from collections import defaultdict

class PreTrainingValidator:
    """
    Comprehensive validation before training 21 models
    Tests: Data, Models, Training Functions, GPU, Stability
    """
    
    def __init__(self, train_loader, val_loader, test_loader, device):
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device
        self.test_results = defaultdict(list)
        self.passed_tests = 0
        self.total_tests = 0
        
    def run_all_tests(self):
        """Run complete test suite"""
        
        print("\n" + "="*80)
        print("🧪 COMPREHENSIVE PRE-TRAINING VALIDATION")
        print("="*80 + "\n")
        
        # Run all test categories
        self.test_environment()
        self.test_data_loaders()
        self.test_all_models()
        self.test_training_functions()
        self.test_loss_functions()
        self.test_stability()
        
        # Print final summary
        self.print_summary()
        
        return self.passed_tests == self.total_tests
    
    def _run_test(self, test_name, test_func):
        """Helper to run individual test with error handling"""
        self.total_tests += 1
        try:
            result = test_func()
            if result:
                self.passed_tests += 1
                print(f"   ✅ {test_name}")
                self.test_results[test_name] = "PASS"
                return True
            else:
                print(f"   ❌ {test_name}: FAILED")
                self.test_results[test_name] = "FAIL"
                return False
        except Exception as e:
            print(f"   ❌ {test_name}: ERROR - {str(e)[:60]}")
            self.test_results[test_name] = f"ERROR: {str(e)[:60]}"
            return False
    
    # ========================================================================
    # TEST CATEGORY 1: ENVIRONMENT
    # ========================================================================
    
    def test_environment(self):
        """Test computational environment"""
        
        print("1️⃣  ENVIRONMENT TESTS")
        print("-" * 80)
        
        def test_cuda():
            if self.device.type == 'cuda':
                torch.cuda.empty_cache()
                memory = torch.cuda.get_device_properties(0).total_memory / 1e9
                return memory > 4  # At least 4GB GPU
            return True  # CPU is ok
        
        def test_pytorch_version():
            version = torch.__version__
            major, minor = int(version.split('.')[0]), int(version.split('.')[1])
            return major >= 1 and minor >= 10
        
        def test_device_available():
            return self.device is not None
        
        def test_amp_support():
            if self.device.type == 'cuda':
                return torch.cuda.get_device_capability()[0] >= 7
            return True
        
        self._run_test("CUDA/Device Available", test_device_available)
        self._run_test("PyTorch Version (>=1.10)", test_pytorch_version)
        self._run_test("GPU Memory (>4GB)", test_cuda)
        self._run_test("Mixed Precision Support", test_amp_support)
        
        print()
    
    # ========================================================================
    # TEST CATEGORY 2: DATA LOADERS
    # ========================================================================
    
    def test_data_loaders(self):
        """Test all data loaders"""
        
        print("2️⃣  DATA LOADER TESTS")
        print("-" * 80)
        
        def test_train_loader():
            batch = next(iter(self.train_loader))
            images, features, prices = batch
            return (len(batch) == 3 and 
                    images.shape[0] > 0 and 
                    features.shape[0] > 0 and
                    prices.shape[0] > 0)
        
        def test_val_loader():
            batch = next(iter(self.val_loader))
            images, features, prices = batch
            return len(batch) == 3 and images.shape[0] > 0
        
        def test_test_loader():
            batch = next(iter(self.test_loader))
            images, features, prices = batch
            return len(batch) == 3 and images.shape[0] > 0
        
        def test_data_shapes():
            images, features, prices = next(iter(self.train_loader))
            return (len(images.shape) == 4 and  # (B, C, H, W)
                    images.shape[1] == 3 and  # 3 channels
                    len(features.shape) == 2 and  # (B, F)
                    len(prices.shape) == 1)  # (B,)
        
        def test_data_ranges():
            images, features, prices = next(iter(self.train_loader))
            return (images.min() >= 0 and images.max() <= 1 and  # Normalized images
                    torch.isfinite(features).all() and
                    torch.isfinite(prices).all())
        
        def test_no_nan_inf():
            images, features, prices = next(iter(self.train_loader))
            return (not torch.isnan(images).any() and
                    not torch.isnan(features).any() and
                    not torch.isnan(prices).any() and
                    not torch.isinf(images).any() and
                    not torch.isinf(features).any() and
                    not torch.isinf(prices).any())
        
        def test_batch_consistency():
            batch1 = next(iter(self.train_loader))
            batch2 = next(iter(self.train_loader))
            return batch1[0].shape == batch2[0].shape
        
        def test_dataset_sizes():
            return (len(self.train_loader.dataset) >= 100 and
                    len(self.val_loader.dataset) >= 20 and
                    len(self.test_loader.dataset) >= 20)
        
        self._run_test("Train Loader Functional", test_train_loader)
        self._run_test("Validation Loader Functional", test_val_loader)
        self._run_test("Test Loader Functional", test_test_loader)
        self._run_test("Data Shapes Correct", test_data_shapes)
        self._run_test("Data Ranges Valid", test_data_ranges)
        self._run_test("No NaN/Inf in Data", test_no_nan_inf)
        self._run_test("Batch Consistency", test_batch_consistency)
        self._run_test("Dataset Sizes Adequate", test_dataset_sizes)
        
        # Print data info
        images, features, prices = next(iter(self.train_loader))
        print(f"\n   📊 Data Info:")
        print(f"      Images: {images.shape}")
        print(f"      Features: {features.shape} (num_features={features.shape[1]})")
        print(f"      Prices: {prices.shape}")
        print(f"      Train samples: {len(self.train_loader.dataset)}")
        print(f"      Val samples: {len(self.val_loader.dataset)}")
        print(f"      Test samples: {len(self.test_loader.dataset)}")
        print()
    
    # ========================================================================
    # TEST CATEGORY 3: MODEL ARCHITECTURES
    # ========================================================================
    
    def test_all_models(self):
        """Test all 21 model architectures"""
        
        print("3️⃣  MODEL ARCHITECTURE TESTS (21 MODELS)")
        print("-" * 80)
        
        # Get number of features from data
        sample_batch = next(iter(self.train_loader))
        num_features = sample_batch[1].shape[1]
        
        # Define all 21+ models to test (SAFE MODELS ONLY)
        test_models = [
            # EfficientNet Family (5 models)
            'efficientnet_b0',
            'efficientnet_b1', 
            'efficientnet_b2',
            'efficientnet_b3',
            'efficientnet_b4',
            
            # ResNet Family (4 models)
            'resnet50',
            'resnet101',
            'resnet152',
            'wide_resnet50_2',
            
            # ConvNeXt Family (3 models)
            'convnext_tiny',
            'convnext_small',
            'convnext_base',
            
            # DenseNet Family (2 models)
            'densenet121',
            'densenet169',
            
            # MobileNet Family (2 models)
            'mobilenet_v2',
            'mobilenet_v3_large',
            
            # Others (5 models)
            'regnet_y_400mf',
            'regnet_y_800mf',
            'regnet_y_1_6gf',
            'shufflenet_v2_x1_0',
            'mnasnet1_0'
        ]
        
        successful_models = []
        failed_models = []
        
        for model_name in test_models:
            try:
                # Create model
                model = create_stable_model(
                    backbone_name=model_name,
                    num_features=num_features,
                    fusion_method='concat',
                    dropout=0.3,
                    pretrained=True
                )
                model = model.to(self.device)
                model.eval()
                
                # Test forward pass
                with torch.no_grad():
                    dummy_images = torch.randn(2, 3, 224, 224).to(self.device)
                    dummy_features = torch.randn(2, num_features).to(self.device)
                    output = model(dummy_images, dummy_features)
                
                # Check output validity
                if (torch.isfinite(output).all() and 
                    output.shape[0] == 2 and
                    len(output.shape) == 1):
                    print(f"   ✅ {model_name:30s} (params: {sum(p.numel() for p in model.parameters()):,})")
                    successful_models.append(model_name)
                    self.passed_tests += 1
                else:
                    print(f"   ⚠️  {model_name:30s} - Invalid output")
                    failed_models.append(model_name)
                
                self.total_tests += 1
                
                # Clean up memory
                del model
                torch.cuda.empty_cache() if torch.cuda.is_available() else None
                
            except Exception as e:
                print(f"   ❌ {model_name:30s} - {str(e)[:40]}")
                failed_models.append(model_name)
                self.total_tests += 1
        
        print(f"\n   📊 Model Test Summary:")
        print(f"      ✅ Passed: {len(successful_models)}/{len(test_models)}")
        print(f"      ❌ Failed: {len(failed_models)}/{len(test_models)}")
        
        if len(successful_models) >= 21:
            print(f"      🎉 {len(successful_models)} models available - EXCELLENT!")
        elif len(successful_models) >= 15:
            print(f"      ✅ {len(successful_models)} models available - GOOD")
        else:
            print(f"      ⚠️  Only {len(successful_models)} models available - NEEDS ATTENTION")
        
        print()
        return successful_models
    
    # ========================================================================
    # TEST CATEGORY 4: TRAINING FUNCTIONS
    # ========================================================================
    
    def test_training_functions(self):
        """Test training pipeline components"""
        
        print("4️⃣  TRAINING FUNCTION TESTS")
        print("-" * 80)
        
        def test_loss_functions_exist():
            try:
                loss1 = SafeCombinedLoss()
                loss2 = SafeHuberLoss()
                return True
            except:
                return False
        
        def test_optimizer_creation():
            try:
                model = create_stable_model('efficientnet_b0', num_features=5)
                opt = torch.optim.AdamW(model.parameters(), lr=0.001)
                return opt is not None
            except:
                return False
        
        def test_training_loop_exists():
            return callable(train_ultra_stable)
        
        def test_ensemble_trainer_exists():
            try:
                trainer = EnsembleTrainer()
                return trainer is not None
            except:
                return False
        
        def test_gradient_clipping():
            try:
                model = create_stable_model('efficientnet_b0', num_features=5)
                model = model.to(self.device)
                
                # Create dummy batch
                images = torch.randn(2, 3, 224, 224, requires_grad=True).to(self.device)
                features = torch.randn(2, 5, requires_grad=True).to(self.device)
                
                output = model(images, features)
                loss = output.mean()
                loss.backward()
                
                # Test gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                
                return True
            except:
                return False
        
        self._run_test("Loss Functions Available", test_loss_functions_exist)
        self._run_test("Optimizer Creation", test_optimizer_creation)
        self._run_test("Training Loop Available", test_training_loop_exists)
        self._run_test("Ensemble Trainer Available", test_ensemble_trainer_exists)
        self._run_test("Gradient Clipping Works", test_gradient_clipping)
        
        print()
    
    # ========================================================================
    # TEST CATEGORY 5: LOSS FUNCTIONS
    # ========================================================================
    
    def test_loss_functions(self):
        """Test all loss functions for stability"""
        
        print("5️⃣  LOSS FUNCTION TESTS")
        print("-" * 80)
        
        # Test data
        pred = torch.randn(10).to(self.device)
        target = torch.randn(10).to(self.device)
        
        def test_mse_loss():
            criterion = nn.MSELoss()
            loss = criterion(pred, target)
            return torch.isfinite(loss).item()
        
        def test_mae_loss():
            criterion = nn.L1Loss()
            loss = criterion(pred, target)
            return torch.isfinite(loss).item()
        
        def test_huber_loss():
            criterion = SafeHuberLoss()
            loss = criterion(pred, target)
            return torch.isfinite(loss).item()
        
        def test_combined_loss():
            criterion = SafeCombinedLoss()
            loss = criterion(pred, target)
            return torch.isfinite(loss).item()
        
        def test_extreme_values():
            extreme_pred = torch.tensor([1e10, -1e10, float('nan')]).to(self.device)
            extreme_target = torch.tensor([1.0, 1.0, 1.0]).to(self.device)
            criterion = SafeCombinedLoss()
            try:
                loss = criterion(extreme_pred, extreme_target)
                return torch.isfinite(loss).item()
            except:
                return False
        
        self._run_test("MSE Loss", test_mse_loss)
        self._run_test("MAE Loss", test_mae_loss)
        self._run_test("Huber Loss", test_huber_loss)
        self._run_test("Combined Loss", test_combined_loss)
        self._run_test("Loss Handles Extreme Values", test_extreme_values)
        
        print()
    
    # ========================================================================
    # TEST CATEGORY 6: STABILITY
    # ========================================================================
    
    def test_stability(self):
        """Test model stability with mini training run"""
        
        print("6️⃣  STABILITY TESTS (Mini Training)")
        print("-" * 80)
        
        def test_single_epoch_training():
            """Run 1 epoch to test stability"""
            try:
                # Get data dimensions
                sample_batch = next(iter(self.train_loader))
                num_features = sample_batch[1].shape[1]
                
                # Create small model
                model = create_stable_model(
                    backbone_name='efficientnet_b0',
                    num_features=num_features,
                    dropout=0.3
                )
                model = model.to(self.device)
                model.train()
                
                # Setup training
                criterion = SafeCombinedLoss()
                optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
                
                # Train for 5 batches
                losses = []
                for batch_idx, (images, features, prices) in enumerate(self.train_loader):
                    if batch_idx >= 5:
                        break
                    
                    images = images.to(self.device)
                    features = features.to(self.device)
                    prices = prices.to(self.device)
                    
                    optimizer.zero_grad()
                    outputs = model(images, features)
                    loss = criterion(outputs, prices)
                    
                    if not torch.isfinite(loss):
                        return False
                    
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    optimizer.step()
                    
                    losses.append(loss.item())
                
                # Check all losses are finite
                return all(np.isfinite(losses))
                
            except Exception as e:
                print(f"      Error: {str(e)[:50]}")
                return False
        
        def test_validation_pass():
            """Test validation loop"""
            try:
                sample_batch = next(iter(self.train_loader))
                num_features = sample_batch[1].shape[1]
                
                model = create_stable_model(
                    backbone_name='efficientnet_b0',
                    num_features=num_features
                )
                model = model.to(self.device)
                model.eval()
                
                criterion = SafeCombinedLoss()
                
                val_losses = []
                with torch.no_grad():
                    for batch_idx, (images, features, prices) in enumerate(self.val_loader):
                        if batch_idx >= 3:
                            break
                        
                        images = images.to(self.device)
                        features = features.to(self.device)
                        prices = prices.to(self.device)
                        
                        outputs = model(images, features)
                        loss = criterion(outputs, prices)
                        
                        if not torch.isfinite(loss):
                            return False
                        
                        val_losses.append(loss.item())
                
                return all(np.isfinite(val_losses))
                
            except:
                return False
        
        def test_gradient_flow():
            """Test gradients are flowing"""
            try:
                sample_batch = next(iter(self.train_loader))
                num_features = sample_batch[1].shape[1]
                
                model = create_stable_model('efficientnet_b0', num_features=num_features)
                model = model.to(self.device)
                
                images, features, prices = sample_batch
                images = images.to(self.device, dtype=torch.float32)
                features = features.to(self.device, dtype=torch.float32)
                prices = prices.to(self.device, dtype=torch.float32)
                
                outputs = model(images, features)
                loss = outputs.mean()
                loss.backward()
                
                # Check if gradients exist and are finite
                has_grads = False
                for param in model.parameters():
                    if param.grad is not None and torch.isfinite(param.grad).all():
                        has_grads = True
                        break
                
                return has_grads
                
            except Exception as e:
                print(f"      Error: {str(e)[:50]}")
                return False
        
        self._run_test("Single Epoch Training", test_single_epoch_training)
        self._run_test("Validation Pass", test_validation_pass)
        self._run_test("Gradient Flow", test_gradient_flow)
        
        print()
    
    # ========================================================================
    # SUMMARY
    # ========================================================================
    
    def print_summary(self):
        """Print comprehensive test summary"""
        
        print("\n" + "="*80)
        print("📊 TEST SUMMARY")
        print("="*80)
        
        pass_rate = (self.passed_tests / self.total_tests * 100) if self.total_tests > 0 else 0
        
        print(f"\n✅ Passed: {self.passed_tests}/{self.total_tests} tests ({pass_rate:.1f}%)")
        print(f"❌ Failed: {self.total_tests - self.passed_tests}/{self.total_tests} tests")
        
        if pass_rate == 100:
            print(f"\n🎉 ALL TESTS PASSED! Ready to train 21 models!")
            print(f"   System is STABLE and READY for production training.")
        elif pass_rate >= 90:
            print(f"\n✅ MOSTLY READY ({pass_rate:.0f}% pass rate)")
            print(f"   Minor issues detected but training should work.")
        elif pass_rate >= 75:
            print(f"\n⚠️  NEEDS ATTENTION ({pass_rate:.0f}% pass rate)")
            print(f"   Some issues detected. Review failures before training.")
        else:
            print(f"\n❌ NOT READY ({pass_rate:.0f}% pass rate)")
            print(f"   Critical issues detected. Fix errors before training.")
        
        # Print failed tests
        failed_tests = [name for name, result in self.test_results.items() 
                       if result not in ["PASS", "N/A"]]
        
        if failed_tests:
            print(f"\n⚠️  Failed Tests:")
            for test in failed_tests:
                print(f"   - {test}: {self.test_results[test]}")
        
        print("\n" + "="*80 + "\n")


# ============================================================================
# COMPLETE 21 MODEL TRAINING PIPELINE
# ============================================================================

def train_21_robust_models(train_loader, val_loader, test_loader, device='cuda'):
    """
    Complete pipeline: Test → Train 21 Models → Evaluate
    """
    
    print("\n" + "🏠"*40)
    print("HOUSE PRICE PREDICTION - 21 MODEL ENSEMBLE")
    print("🏠"*40 + "\n")
    
    # ========================================================================
    # STEP 1: PRE-TRAINING VALIDATION
    # ========================================================================
    
    print("STEP 1: PRE-TRAINING VALIDATION")
    print("="*80 + "\n")
    
    validator = PreTrainingValidator(train_loader, val_loader, test_loader, device)
    all_tests_passed = validator.run_all_tests()
    
    if not all_tests_passed:
        response = input("\n⚠️  Some tests failed. Continue anyway? (yes/no): ")
        if response.lower() != 'yes':
            print("❌ Training cancelled. Please fix errors and try again.")
            return None, None, None
    
    # ========================================================================
    # STEP 2: DEFINE 21+ MODELS
    # ========================================================================
    
    print("\nSTEP 2: DEFINING 21 MODEL ARCHITECTURES")
    print("="*80 + "\n")
    
    model_names = [
        # EfficientNet Family (5 models) - Best for house images
        'efficientnet_b0',
        'efficientnet_b1',
        'efficientnet_b2',
        'efficientnet_b3',
        'efficientnet_b4',
        
        # ResNet Family (4 models) - Stable baseline
        'resnet50',
        'resnet101',
        'resnet152',
        'wide_resnet50_2',
        
        # ConvNeXt (3 models) - Modern architecture
        'convnext_tiny',
        'convnext_small',
        'convnext_base',
        
        # DenseNet (2 models) - Dense connections
        'densenet121',
        'densenet169',
        
        # MobileNet (2 models) - Efficient
        'mobilenet_v2',
        'mobilenet_v3_large',
        
        # RegNet (3 models) - Regularized networks
        'regnet_y_400mf',
        'regnet_y_800mf',
        'regnet_y_1_6gf',
        
        # Others (2 models)
        'shufflenet_v2_x1_0',
        'mnasnet1_0'
    ]
    
    print(f"📋 Total models to train: {len(model_names)}")
    print(f"📦 Models: {', '.join(model_names[:5])}... (and {len(model_names)-5} more)")
    
    # Estimate training time
    avg_time_per_model = 10  # minutes (estimate)
    total_time_estimate = len(model_names) * avg_time_per_model
    
    print(f"\n⏱️  Estimated training time:")
    print(f"   Per model: ~{avg_time_per_model} minutes")
    print(f"   Total: ~{total_time_estimate} minutes ({total_time_estimate/60:.1f} hours)")
    
    response = input(f"\n🚀 Ready to train {len(model_names)} models? (yes/no): ")
    if response.lower() != 'yes':
        print("❌ Training cancelled.")
        return None, None, None
    
    # ========================================================================
    # STEP 3: TRAIN ENSEMBLE
    # ========================================================================
    
    print("\n\nSTEP 3: TRAINING ENSEMBLE")
    print("="*80 + "\n")
    
    trained_models, all_results, trainer = run_ensemble_training(
        model_names=model_names,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        device=device
    )
    
    # ========================================================================
    # STEP 4: FINAL ANALYSIS
    # ========================================================================
    
    print("\n\nSTEP 4: FINAL ANALYSIS")
    print("="*80 + "\n")
    
    if len(all_results) > 0:
        # Find best model
        valid_results = [r for r in all_results if r.get('test_r2', 0) > 0]
        
        if len(valid_results) > 0:
            best_model = max(valid_results, key=lambda x: x['test_r2'])
            
            print(f"🏆 BEST MODEL: {best_model['model_name']}")
            print(f"   R² Score: {best_model['test_r2']:.4f}")
            print(f"   RMSE: ${best_model['test_rmse']:,.0f}")
            print(f"   MAE: ${best_model['test_mae']:,.0f}")
            print(f"   Training Time: {best_model['training_time']/60:.2f} min")
            
            # Ensemble prediction (average of top 5 models)
            top_5 = sorted(valid_results, key=lambda x: x['test_r2'], reverse=True)[:5]
            avg_r2 = np.mean([m['test_r2'] for m in top_5])
            avg_rmse = np.mean([m['test_rmse'] for m in top_5])
            
            print(f"\n🔝 TOP 5 ENSEMBLE:")
            print(f"   Average R²: {avg_r2:.4f}")
            print(f"   Average RMSE: ${avg_rmse:,.0f}")
            
            print(f"\n✅ Training complete! Check '{trainer.session_dir}' for all outputs.")
    
    print("\n" + "="*80)
    print("🎉 ENSEMBLE TRAINING PIPELINE COMPLETE")
    print("="*80 + "\n")
    
    return trained_models, all_results, trainer


# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    """
    Complete execution:
    1. Run pre-training tests
    2. Train 21 models
    3. Generate comprehensive analysis
    """
    
    # Assuming you have these defined from previous stages:
    # - train_loader
    # - val_loader  
    # - test_loader
    # - device
    
    trained_models, all_results, trainer = train_21_robust_models(
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        device=device
    )
    
    print("\n✅ ALL DONE! Check the output directory for:")
    print("   - Model checkpoints")
    print("   - Training logs")
    print("   - Performance visualizations")
    print("   - JSON results for each model")


📁 Session Directory: model_outputs\20251126_030804

🚀 Starting Ensemble Training Session
📊 Total models to train: 21
📁 Session ID: 20251126_030804
💾 Output directory: model_outputs\20251126_030804
🖥️  Device: cuda


################################################################################
# [1/21] 🏗️  Training: ResNet50_MultiModal
################################################################################

📋 Configuration for ResNet50_MultiModal:
   lr: 0.001
   num_epochs: 50
   loss_type: huber
   optimizer_type: adamw
   scheduler_type: cosine
   use_swa: True
   use_mixup: True
   gradient_accumulation_steps: 1


🚀 Training ResNet50_MultiModal with ROBUST Configuration
✓ Number of features: 5
📦 Creating model architecture: ResNet50_MultiModal...
❌ Error creating model: name 'get_model' is not defined

❌ Error training ResNet50_MultiModal: name 'get_model' is not defined

────────────────────────────────────────────────────────────────────────────────
📈 Progress: 1/21 mo

Traceback (most recent call last):
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_20916\2782451258.py", line 192, in train_single_model
    model, results = train_model_robust(
                     ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_20916\475955486.py", line 109, in train_model_robust
    model = get_model(model_name, num_features=num_features).to(device)
            ^^^^^^^^^
NameError: name 'get_model' is not defined. Did you mean: 'test_models'?
Traceback (most recent call last):
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_20916\2782451258.py", line 192, in train_single_model
    model, results = train_model_robust(
                     ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_20916\475955486.py", line 109, in train_model_robust
    model = get_model(model_name, num_features=num_features).to(device)
            ^^^^^^^^^
NameError: name 'get_model' is not defined. Did you mean: 'test_models'?
Traceb

✓ Number of features: 5
📦 Creating model architecture: MobileNetV2_MultiModal...
❌ Error creating model: name 'get_model' is not defined

❌ Error training MobileNetV2_MultiModal: name 'get_model' is not defined

────────────────────────────────────────────────────────────────────────────────
📈 Progress: 9/21 models completed
✅ Successful: 0 | ❌ Failed: 9
⏱️  Elapsed: 0.0min | ETA: 0.0min
────────────────────────────────────────────────────────────────────────────────


################################################################################
# [10/21] 🏗️  Training: MobileNetV3_MultiModal
################################################################################

📋 Configuration for MobileNetV3_MultiModal:
   lr: 0.004
   num_epochs: 45
   loss_type: huber
   optimizer_type: adamw
   scheduler_type: onecycle
   use_swa: False
   use_mixup: False
   gradient_accumulation_steps: 1


🚀 Training MobileNetV3_MultiModal with ROBUST Configuration
✓ Number of features: 5
📦 Creating

Traceback (most recent call last):
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_20916\2782451258.py", line 192, in train_single_model
    model, results = train_model_robust(
                     ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_20916\475955486.py", line 109, in train_model_robust
    model = get_model(model_name, num_features=num_features).to(device)
            ^^^^^^^^^
NameError: name 'get_model' is not defined. Did you mean: 'test_models'?
Traceback (most recent call last):
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_20916\2782451258.py", line 192, in train_single_model
    model, results = train_model_robust(
                     ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_20916\475955486.py", line 109, in train_model_robust
    model = get_model(model_name, num_features=num_features).to(device)
            ^^^^^^^^^
NameError: name 'get_model' is not defined. Did you mean: 'test_models'?
Traceb

✓ Number of features: 5
📦 Creating model architecture: ConvNeXt_MultiModal...
❌ Error creating model: name 'get_model' is not defined

❌ Error training ConvNeXt_MultiModal: name 'get_model' is not defined

────────────────────────────────────────────────────────────────────────────────
📈 Progress: 18/21 models completed
✅ Successful: 0 | ❌ Failed: 18
⏱️  Elapsed: 0.0min | ETA: 0.0min
────────────────────────────────────────────────────────────────────────────────


################################################################################
# [19/21] 🏗️  Training: Swin_Transformer_MultiModal
################################################################################

📋 Configuration for Swin_Transformer_MultiModal:
   lr: 0.003
   num_epochs: 50
   loss_type: huber
   optimizer_type: adamw
   scheduler_type: cosine
   use_swa: True
   use_mixup: True
   gradient_accumulation_steps: 1


🚀 Training Swin_Transformer_MultiModal with ROBUST Configuration
✓ Number of features: 5
📦 C

Traceback (most recent call last):
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_20916\2782451258.py", line 192, in train_single_model
    model, results = train_model_robust(
                     ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_20916\475955486.py", line 109, in train_model_robust
    model = get_model(model_name, num_features=num_features).to(device)
            ^^^^^^^^^
NameError: name 'get_model' is not defined. Did you mean: 'test_models'?
Traceback (most recent call last):
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_20916\2782451258.py", line 192, in train_single_model
    model, results = train_model_robust(
                     ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_20916\475955486.py", line 109, in train_model_robust
    model = get_model(model_name, num_features=num_features).to(device)
            ^^^^^^^^^
NameError: name 'get_model' is not defined. Did you mean: 'test_models'?
Traceb

In [12]:
# ============================================================================
# RESTART TRAINING - With All Stability Fixes
# ============================================================================

import torch

print("="*80)
print("🔄 RESTARTING TRAINING WITH STABILITY FIXES")
print("="*80)

# ============================================================================
# STEP 1: Update get_model function
# ============================================================================

# Option A: Quick patch - replace get_model with get_model_stable
import types

original_get_model = get_model

def patched_get_model(model_name, num_features=5, **kwargs):
    """Patched version that uses StableHousePriceModel"""
    try:
        return get_model_stable(model_name, num_features)
    except:
        # Fallback to original if stable version fails
        return original_get_model(model_name, num_features, **kwargs)

# Replace globally
get_model = patched_get_model
print("✅ Patched get_model to use stable version")

# ============================================================================
# STEP 2: Use CONSERVATIVE model list (best performers)
# ============================================================================

# Start with most stable models
conservative_models = [
    'ResNet50',           # Very stable
    'ResNet101',          # Very stable
    'DenseNet121',        # Stable
    'DenseNet169',        # Stable
    'EfficientNetB0',     # Stable
    'EfficientNetB3',     # Moderate
    'MobileNetV2',        # Stable
    'VGG16',              # Stable but slow
]

print(f"\n📊 Training {len(conservative_models)} conservative models")
print("   These models are less likely to have NaN/Inf issues")

# ============================================================================
# STEP 3: Train with monitoring
# ============================================================================

def train_with_monitoring(model_names, max_failures=3):
    """Train models with failure monitoring"""
    
    successful_models = []
    failed_models = []
    
    trained_models, all_results, trainer = run_ensemble_training(
        model_names=model_names,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        device=device
    )
    
    # Check results
    for result in all_results:
        if result.get('test_r2', 0) > 0.5:  # Reasonable R²
            successful_models.append(result['model_name'])
        else:
            failed_models.append(result['model_name'])
    
    print(f"\n📊 Training Summary:")
    print(f"   ✅ Successful: {len(successful_models)}")
    print(f"   ❌ Failed/Poor: {len(failed_models)}")
    
    if len(successful_models) > 0:
        print(f"\n🏆 Good models:")
        for name in successful_models:
            print(f"      - {name}")
    
    if len(failed_models) > 0:
        print(f"\n⚠️  Problematic models:")
        for name in failed_models:
            print(f"      - {name}")
    
    return trained_models, all_results, trainer

# ============================================================================
# STEP 4: Run training
# ============================================================================

print("\n🚀 Starting training...")
print("="*80)

trained_models, all_results, trainer = train_with_monitoring(
    model_names=conservative_models
)

print("\n" + "="*80)
print("✅ TRAINING COMPLETE")
print("="*80)

# ============================================================================
# STEP 5: Show best results
# ============================================================================

import pandas as pd

if len(all_results) > 0:
    # Filter out failed models
    valid_results = [r for r in all_results if r.get('test_r2', 0) > 0.1]
    
    if len(valid_results) > 0:
        df = pd.DataFrame([{
            'Model': r['model_name'],
            'R² Score': r.get('test_r2', 0),
            'RMSE': r.get('test_rmse', float('inf')),
            'MAE': r.get('test_mae', float('inf')),
            'Training Time (min)': r.get('training_time', 0) / 60
        } for r in valid_results])
        
        df = df.sort_values('R² Score', ascending=False)
        
        print("\n🏆 TOP PERFORMING MODELS:")
        print("="*80)
        print(df.to_string(index=False))
        print("="*80)
        
        best_model = df.iloc[0]
        print(f"\n⭐ Best Model: {best_model['Model']}")
        print(f"   R² Score: {best_model['R² Score']:.4f}")
        print(f"   RMSE: ${best_model['RMSE']:,.2f}")
        print(f"   MAE: ${best_model['MAE']:,.2f}")
    else:
        print("\n⚠️  No models achieved good performance (R² > 0.1)")
        print("   This suggests a fundamental issue with:")
        print("   1. Data normalization/scaling")
        print("   2. Price target distribution")
        print("   3. Feature quality")
        
        print("\n🔍 Debugging suggestions:")
        print("   - Check if prices are normalized correctly")
        print("   - Verify image preprocessing")
        print("   - Ensure no NaN in input data")
        print("   - Check if price range is reasonable")
else:
    print("\n❌ No models completed training")

# ============================================================================
# STEP 6: Next steps
# ============================================================================

print("\n" + "="*80)
print("📝 NEXT STEPS:")
print("="*80)

if len([r for r in all_results if r.get('test_r2', 0) > 0.5]) >= 3:
    print("\n✅ Good! You have working models. Now you can:")
    print("   1. Add more models from model_names_diverse")
    print("   2. Create ensemble predictions")
    print("   3. Fine-tune hyperparameters")
else:
    print("\n⚠️  Few/no good models. You should:")
    print("   1. Check data preprocessing code")
    print("   2. Verify price normalization")
    print("   3. Examine a few training samples manually")
    print("   4. Try even lower learning rates (0.0001)")
    
print("\n" + "="*80)

🔄 RESTARTING TRAINING WITH STABILITY FIXES


NameError: name 'get_model' is not defined

In [17]:
# ============================================================================
# COMPLETE INTEGRATION - RUN THIS TO START TRAINING
# ============================================================================

import torch
from torch.utils.data import DataLoader

# ============================================================================
# STEP 1: Choose Your Model List
# ============================================================================

# Option A: Use your original 21 model names (compatible with old code)
model_names = [
    'EfficientNet', 'MobileNet-v2', 'ResNet', 'DenseNet', 'Xception',
    'Inception-V3', 'GoogleNet', 'VGG', 'Squeeze-and-Excitation',
    'Residual-Attention', 'WideResNet', 'Inception-ResNet-v2',
    'Inception-V4', 'Competitive-SE', 'HRNetV2', 'FractalNet',
    'Highway', 'AlexNet', 'NIN', 'ZFNet', 'CapsuleNet'
]

# Option B: Use extended model names for better diversity (RECOMMENDED)
model_names_diverse = [
    # ResNet family (3 models)
    'ResNet50', 'ResNet101', 'ResNet152',
    
    # EfficientNet family (4 models)
    'EfficientNetB0', 'EfficientNetB3', 'EfficientNetB5', 'EfficientNetV2',
    
    # DenseNet family (3 models)
    'DenseNet121', 'DenseNet169', 'DenseNet201',
    
    # MobileNet family (2 models)
    'MobileNetV2', 'MobileNetV3Large',
    
    # VGG family (2 models)
    'VGG16', 'VGG19',
    
    # Vision Transformers (2 models)
    'ViT_B16', 'ViT_B32',
    
    # Modern architectures (4 models)
    'SwinTransformer', 'ConvNeXt', 'RegNetY', 'WideResNet',
    
    # Classic (1 model)
    'Inception-V3'
]  # Total: 21 models

print(f'📊 Total models to train: {len(model_names_diverse)}')

# ============================================================================
# STEP 2: Run Complete Training Pipeline
# ============================================================================

# Make sure you have:
# - train_loader (DataLoader for training data)
# - val_loader (DataLoader for validation data)
# - test_loader (DataLoader for test data)
# - device (cuda or cpu)

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'🖥️  Device: {device}')

# Run ensemble training with the improved pipeline
trained_models, all_results, trainer = run_ensemble_training(
    model_names=model_names_diverse,  # Or use model_names for original
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    device=device
)

# ============================================================================
# STEP 3: Access Results
# ============================================================================

print('\n' + '='*80)
print('📊 TRAINING COMPLETE - RESULTS SUMMARY')
print('='*80)

# Get session directory
session_dir = trainer.session_dir
print(f'📁 All outputs saved to: {session_dir}')

# Load summary CSV
import pandas as pd
summary_df = pd.read_csv(session_dir / 'training_summary.csv')

print('\n🏆 Top 5 Models by R² Score:')
print(summary_df.head(5)[['Model', 'Test R²', 'Test RMSE', 'Training Time (min)']].to_string(index=False))

# Best model
best_model_name = summary_df.iloc[0]['Model']
best_r2 = summary_df.iloc[0]['Test R²']
best_rmse = summary_df.iloc[0]['Test RMSE']

print(f'\n⭐ Best Model: {best_model_name}')
print(f'   R² Score: {best_r2:.4f}')
print(f'   RMSE: ${best_rmse:,.2f}')

# ============================================================================
# STEP 4: Create Ensemble Predictions (Optional)
# ============================================================================

def create_ensemble_predictions(trained_models, test_loader, top_k=10):
    """
    Create ensemble predictions using top K models
    """
    
    print(f'\n🔮 Creating ensemble predictions using top {top_k} models...')
    
    # Get top K model names
    top_models = summary_df.head(top_k)['Model'].tolist()
    
    all_predictions = []
    
    for model_name in top_models:
        if model_name in trained_models:
            model = trained_models[model_name]
            model.eval()
            
            batch_predictions = []
            
            with torch.no_grad():
                for images, features, _ in test_loader:
                    images = images.to(device)
                    features = features.to(device)
                    
                    preds = model(images, features)
                    batch_predictions.extend(preds.cpu().numpy())
            
            all_predictions.append(batch_predictions)
    
    # Average predictions
    import numpy as np
    ensemble_preds = np.mean(all_predictions, axis=0)
    
    print(f'✅ Ensemble predictions created using {len(all_predictions)} models')
    
    return ensemble_preds

# Create ensemble predictions
ensemble_predictions = create_ensemble_predictions(
    trained_models=trained_models,
    test_loader=test_loader,
    top_k=10  # Use top 10 models
)

# ============================================================================
# STEP 5: Evaluate Ensemble Performance
# ============================================================================

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# Get actual test targets
test_targets = []
with torch.no_grad():
    for _, _, prices in test_loader:
        test_targets.extend(prices.numpy())

test_targets = np.array(test_targets)

# Denormalize predictions and targets
test_preds_norm = ensemble_predictions.reshape(-1, 1)
test_targets_norm = test_targets.reshape(-1, 1)

test_preds = test_loader.dataset.price_scaler.inverse_transform(test_preds_norm).flatten()
test_targets_real = test_loader.dataset.price_scaler.inverse_transform(test_targets_norm).flatten()

# Calculate metrics
ensemble_r2 = r2_score(test_targets_real, test_preds)
ensemble_rmse = np.sqrt(mean_squared_error(test_targets_real, test_preds))
ensemble_mae = mean_absolute_error(test_targets_real, test_preds)

print('\n' + '='*80)
print('🎯 ENSEMBLE PERFORMANCE (Top 10 Models Average)')
print('='*80)
print(f'R² Score:  {ensemble_r2:.4f}')
print(f'RMSE:      ${ensemble_rmse:,.2f}')
print(f'MAE:       ${ensemble_mae:,.2f}')
print('='*80)

# Compare with best single model
improvement = ((ensemble_r2 - best_r2) / best_r2) * 100
print(f'\n📈 Improvement over best single model: {improvement:+.2f}%')

# ============================================================================
# STEP 6: Visualize Predictions
# ============================================================================

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot
axes[0].scatter(test_targets_real, test_preds, alpha=0.5, s=20)
axes[0].plot([test_targets_real.min(), test_targets_real.max()], 
             [test_targets_real.min(), test_targets_real.max()], 
             'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Price ($)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Predicted Price ($)', fontsize=12, fontweight='bold')
axes[0].set_title(f'Ensemble Predictions (R²={ensemble_r2:.4f})', 
                  fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residuals
residuals = test_targets_real - test_preds
axes[1].scatter(test_preds, residuals, alpha=0.5, s=20)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Price ($)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Residuals ($)', fontsize=12, fontweight='bold')
axes[1].set_title('Residual Plot', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
prediction_plot_path = session_dir / 'ensemble_predictions.png'
plt.savefig(prediction_plot_path, dpi=300, bbox_inches='tight')
print(f'\n📊 Prediction plots saved to: {prediction_plot_path}')
plt.show()

print('\n✅ ALL DONE! 🎉')
print(f'📁 Check {session_dir} for all outputs')

📊 Total models to train: 21
🖥️  Device: cuda
📁 Session Directory: model_outputs\20251126_014813

🚀 Starting Ensemble Training Session
📊 Total models to train: 21
📁 Session ID: 20251126_014813
💾 Output directory: model_outputs\20251126_014813
🖥️  Device: cuda


################################################################################
# [1/21] 🏗️  Training: ResNet50
################################################################################

📋 Configuration for ResNet50:
   lr: 0.003
   num_epochs: 50
   loss_type: huber
   optimizer_type: adamw
   scheduler_type: cosine
   use_swa: True
   use_mixup: True
   gradient_accumulation_steps: 1


🚀 Training ResNet50 with ROBUST Configuration
✓ Number of features: 5
📦 Creating model architecture: ResNet50...
✅ Model created successfully (0.30s)
   📊 Total parameters: 26,121,667
   🎯 Trainable parameters: 26,121,667
   💾 Model size: 99.65 MB

🎯 Loss Function: HUBER
⚙️  Optimizer: ADAMW
📈 Scheduler: COSINE
🔄 Stochastic Weight Averagi

Epoch 1/50 [Train]:   0%|                                                                        | 0/36 [00:00<?, ?it/s]


❌ Error training ResNet50: mat1 and mat2 shapes cannot be multiplied (65536x1 and 2048x128)

────────────────────────────────────────────────────────────────────────────────
📈 Progress: 1/21 models completed
✅ Successful: 0 | ❌ Failed: 1
⏱️  Elapsed: 0.0min | ETA: 0.1min
────────────────────────────────────────────────────────────────────────────────


################################################################################
# [2/21] 🏗️  Training: ResNet101
################################################################################

📋 Configuration for ResNet101:
   lr: 0.003
   num_epochs: 50
   loss_type: combined
   optimizer_type: radam
   scheduler_type: plateau
   use_swa: True
   use_mixup: False
   gradient_accumulation_steps: 1


🚀 Training ResNet101 with ROBUST Configuration
✓ Number of features: 5
📦 Creating model architecture: ResNet101...



Traceback (most recent call last):
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_19752\4108819757.py", line 192, in train_single_model
    model, results = train_model_robust(
                     ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_19752\2022634056.py", line 279, in train_model_robust
    outputs = model(mixed_images, mixed_features)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LEGION\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LEGION\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py", line 1762, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_19752\479995792.py", line 287, in forward
    attention_weights =

✅ Model created successfully (0.36s)
   📊 Total parameters: 45,113,795
   🎯 Trainable parameters: 45,113,795
   💾 Model size: 172.10 MB

🎯 Loss Function: COMBINED
⚙️  Optimizer: RADAM
📈 Scheduler: PLATEAU

❌ Error training ResNet101: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'

────────────────────────────────────────────────────────────────────────────────
📈 Progress: 2/21 models completed
✅ Successful: 0 | ❌ Failed: 2
⏱️  Elapsed: 0.0min | ETA: 0.1min
────────────────────────────────────────────────────────────────────────────────


################################################################################
# [3/21] 🏗️  Training: ResNet152
################################################################################

📋 Configuration for ResNet152:
   lr: 0.003
   num_epochs: 50
   loss_type: combined
   optimizer_type: adamw
   scheduler_type: cosine
   use_swa: True
   use_mixup: False
   gradient_accumulation_steps: 1


🚀 Training ResNet152 wit

...


❌ Error training RegNetY: mat1 and mat2 shapes cannot be multiplied (14080x1 and 440x27)

────────────────────────────────────────────────────────────────────────────────
📈 Progress: 19/21 models completed
✅ Successful: 0 | ❌ Failed: 19
⏱️  Elapsed: 1.1min | ETA: 0.1min
────────────────────────────────────────────────────────────────────────────────


################################################################################
# [20/21] 🏗️  Training: WideResNet
################################################################################

📋 Configuration for WideResNet:
   lr: 0.003
   num_epochs: 50
   loss_type: combined
   optimizer_type: radam
   scheduler_type: plateau
   use_swa: True
   use_mixup: False
   gradient_accumulation_steps: 1


🚀 Training WideResNet with ROBUST Configuration
✓ Number of features: 5
📦 Creating model architecture: WideResNet...
Downloading: "https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth" to C:\Users\LEGION/.cache\torch\hub\che

100%|██████████| 132M/132M [00:17<00:00, 7.98MB/s] 


✅ Model created successfully (19.11s)
   📊 Total parameters: 69,447,875
   🎯 Trainable parameters: 69,447,875
   💾 Model size: 264.92 MB

🎯 Loss Function: COMBINED
⚙️  Optimizer: RADAM
📈 Scheduler: PLATEAU

❌ Error training WideResNet: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'

────────────────────────────────────────────────────────────────────────────────
📈 Progress: 20/21 models completed
✅ Successful: 0 | ❌ Failed: 20
⏱️  Elapsed: 1.5min | ETA: 0.1min
────────────────────────────────────────────────────────────────────────────────


################################################################################
# [21/21] 🏗️  Training: Inception-V3
################################################################################

📋 Configuration for Inception-V3:
   lr: 0.003
   num_epochs: 50
   loss_type: combined
   optimizer_type: adamw
   scheduler_type: cosine
   use_swa: True
   use_mixup: False
   gradient_accumulation_steps: 1


🚀 Training In

Traceback (most recent call last):
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_19752\4108819757.py", line 192, in train_single_model
    model, results = train_model_robust(
                     ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_19752\2022634056.py", line 172, in train_model_robust
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'
Traceback (most recent call last):
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_19752\4108819757.py", line 192, in train_single_model
    model, results = train_model_robust(
                     ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEGION\AppData\Local\Temp\ipykernel_19752\2022634056.py", line 109, in train_model_robust
    model = get_model(model_name, num_features=num_features).to(device)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\U

FileNotFoundError: [Errno 2] No such file or directory: 'model_outputs\\20251126_014813\\training_summary.csv'